In [79]:
import pandas as pd
import numpy as np

from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import (
    train_test_split, StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV
)
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier

from os import cpu_count

n_jobs = max(cpu_count()-1, 1)

In [2]:
train = pd.read_csv('../data/modulbank/train.csv', sep = '\t', index_col=0)
test =pd.read_csv('../data/modulbank/test.csv', sep = '\t', index_col=0)
train.head()

,0,1,2,3,4,5,6,7,8,9,...,336,337,338,339,340,341,342,343,344,345
0,1,1,0,0,0,0,0,0.090909,0,0,...,0.221395,0,1,0,0,0.222222,1,1,1,1
1,1,1,0,0,1,0,0,0.090909,0,0,...,0.241508,0,1,0,0,0.111111,1,1,1,0
2,0,1,0,0,1,0,0,0.090909,0,0,...,0.123067,0,1,0,0,0.444444,1,1,1,1
3,0,1,0,0,1,0,0,0.136364,0,0,...,0.296065,0,0,1,0,0.222222,1,1,1,0
4,0,1,0,0,1,0,0,0.136364,0,0,...,0.178956,0,0,1,0,0.111111,1,1,1,1


In [3]:
target = train['0']

In [4]:
# drop constant columns
unique_value = train.nunique()
one_value_col = unique_value[unique_value == 1].index
unique_value_test = test.nunique()
one_value_col_test = unique_value[unique_value == 1].index
train = train.drop(one_value_col, axis=1, errors='ignore')
test = test.drop(one_value_col_test, axis=1, errors='ignore')

In [5]:
df = pd.concat([train, test])

In [6]:
def get_categorical_col(df, k = 0.9, n = 10):
    #get categorical values
    categ_col = []
    row_num = df.shape[0]
    unique_value = df.nunique()
    columns = unique_value[unique_value <= n].index
    for c in columns:
        if df[c].value_counts().iloc[0] < row_num * k:
            categ_col.append(c)
    return categ_col

In [7]:
categ_col =  get_categorical_col(df, k = 0.9, n = 10)
num_col = list(set(df.columns) - set(categ_col))
categ_col.remove('0')

In [8]:
var_col = df[num_col].var(axis = 0)
num_col_filtered = list(var_col[var_col > 0.1].index)
len(num_col_filtered)

16

In [10]:
import pandas_profiling as pp
import warnings
warnings.filterwarnings('ignore')
pp.ProfileReport(train[categ_col])

Number of variables,38
Number of observations,30500
Total Missing (%),0.0%
Total size in memory,9.1 MiB
Average record size in memory,312.0 B
Numeric,37
Categorical,0
Date,0
Text (Unique),0
Rejected,1
Distinct count,5


In [27]:
import pandas_profiling as pp
import warnings
warnings.filterwarnings('ignore')
pp.ProfileReport(test[categ_col])

Number of variables,38
Number of observations,4166
Total Missing (%),0.0%
Total size in memory,1.2 MiB
Average record size in memory,312.0 B
Numeric,37
Categorical,0
Date,0
Text (Unique),0
Rejected,1
Distinct count,5


In [43]:
import pandas_profiling as pp
import warnings
warnings.filterwarnings('ignore')
pp.ProfileReport(train[num_col])

Number of variables,16
Number of observations,30500
Total Missing (%),0.0%
Total size in memory,5.2 MiB
Average record size in memory,179.0 B
Numeric,12
Categorical,0
Date,0
Text (Unique),0
Rejected,4
Distinct count,307


In [9]:
# Find High correlated features
corr_matrix = df.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
high_cor_col = [column for column in upper.columns if any(upper[column] > 0.96)]
high_cor_col

['2',
 '9',
 '122',
 '146',
 '153',
 '165',
 '180',
 '325',
 '326',
 '329',
 '331',
 '333',
 '334',
 '335']

In [10]:
def write_to_submission_file(predicted, id_col, out_file, col = ['_ID_', '_VAL_']):
    predicted = pd.DataFrame(predicted, columns=['_VAL_'], index=id_col)
    predicted['_ID_'] = predicted.index
    predicted.to_csv(out_file, index=False, sep = ',', columns = col)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(train.drop(['0'],  axis=1, errors='ignore'), target,
                                                    test_size=0.2, stratify= target)

In [12]:
%%time
# train at all features
params = {"C": [0.01, 0.05, 0.1, 1, 10]}
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
lr = LogisticRegression( class_weight = 'balanced')
gs = GridSearchCV(
    estimator=lr,  
    param_grid=params,  
    cv=kf,  
    error_score=1,  
    scoring='roc_auc',  
    n_jobs=n_jobs,
    verbose=2,
)
gs.fit( X= X_train, y= y_train)
best_score = gs.best_score_
best_lr = gs.best_estimator_
print(best_score)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=3)]: Done  25 out of  25 | elapsed:  1.8min finished


0.72862566674
Wall time: 1min 56s


In [16]:
y_pred = best_lr.predict(X_test)
score_valid = roc_auc_score(y_pred, y_test)
score_valid

0.59733978978461355

In [14]:
#drop corr value
df = df.drop(high_cor_col,axis=1, errors='ignore')
train_clean = df[df['0'].notnull()]
test_clean = df[df['0'].isnull()]
target_clean = df[df['0'].notnull()]['0']
train_clean = train_clean.drop(['0'],  axis=1, errors='ignore')
X_train_clean, X_test_clean, y_train_clean, y_test_clean = train_test_split(train_clean, target_clean,
                                                    test_size=0.2, stratify= target_clean)
print(train.shape)

(30500, 342)


In [15]:
%%time
# train without high correlated feature
params = {"C": [0.01, 0.05, 0.1, 1, 10]}
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
lr = LogisticRegression( class_weight = 'balanced')
gs = GridSearchCV(
    estimator=lr,  
    param_grid=params,  
    cv=kf,  
    error_score=1,  
    scoring='roc_auc',  
    n_jobs=n_jobs,
    verbose=2,
)
gs.fit( X= X_train_clean, y= y_train_clean)
best_score = gs.best_score_
best_lr_clean = gs.best_estimator_
print(best_score)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=3)]: Done  25 out of  25 | elapsed:  2.5min finished


0.726587720026
Wall time: 2min 34s


In [16]:
y_pred_clean = best_lr_clean.predict(X_test_clean)
score_valid = roc_auc_score(y_pred_clean, y_test_clean)
score_valid

0.60070948111145062

In [17]:
def get_dummies(df, categ_col):
    # et dummies from categorical values
    for c in categ_col:
        records = pd.get_dummies(df[c], prefix=c)
        df.drop(c, axis=1, inplace=True)
        df = pd.concat([df, records], axis=1)
    return df

In [18]:
#code categorical values
df = pd.concat([train, test])
df = get_dummies(df,categ_col)
print(df.shape)


(34666, 405)


In [19]:
train_code = df[df['0'].notnull()]
test_code = df[df['0'].isnull()]
target_code = df[df['0'].notnull()]['0']
train_code = train_code.drop(['0'],  axis=1, errors='ignore')
test_code = test_code.drop(['0'], axis=1, errors='ignore')
X_train_code, X_test_code, y_train_code, y_test_code = train_test_split(train_code, target_code,
                                                    test_size=0.2, stratify= target_code)

In [20]:
%%time
# train at all features + coded categorical features
params = {"C": [0.01, 0.05, 0.1, 1],
           "solver": ["newton-cg", "lbfgs", "liblinear", "sag", "saga"]}
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#SVM = SVC(probability=True, kernel = 'rbf', C = 1, gamma =  0.01)
lr = LogisticRegression( class_weight = 'balanced')
gs = GridSearchCV(
    estimator=lr,  
    param_grid=params,  
    cv=kf,  
    error_score=1,  
    scoring='roc_auc',  
    n_jobs=n_jobs,
    verbose=2,
)
gs.fit( X= X_train_code, y= y_train_code)
best_score = gs.best_score_
best_lr_code = gs.best_estimator_
print(best_score)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:  4.7min
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed: 14.7min finished


0.725298048657
Wall time: 14min 49s


In [28]:
y_pred_code = best_lr_code.predict(X_test_code)
score_valid = roc_auc_score(y_pred_code, y_test_code)
score_valid

0.60425373972098084

In [21]:
best_lr_code.fit(train_code, target_code )

LogisticRegression(C=0.05, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [23]:
y_log_pred =  best_lr_code.predict(test_code)
write_to_submission_file(y_log_pred.astype(int), test_code.index, 'log_submission.csv' )

In [34]:
# get hiqh variance feature
df = pd.concat([train, test])
print(df.shape)
var_col = df[num_col].var(axis = 0)
num_col_filtered = list(var_col[var_col > 0.025].index)
print(len(num_col_filtered))
to_drop = list(set(num_col) - set(num_col_filtered))

(34666, 342)
86


In [35]:
df = df.drop(to_drop, axis = 1)
train_var = df[df['0'].notnull()]
test_var = df[df['0'].isnull()]
target_var = df[df['0'].notnull()]['0']
train_var = train_var.drop(['0'],  axis=1, errors='ignore')
X_train_var, X_test_var, y_train_var, y_test_var = train_test_split(train_var, target_var, test_size=0.2, stratify= target_var)
        

In [36]:
%%time
# train at features with high variance
params = {"C": [ 0.05, 0.1, 1, 2, 4],
           "solver": ["newton-cg", "lbfgs", "liblinear"]}
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#SVM = SVC(probability=True, kernel = 'rbf', C = 1, gamma =  0.01)
lr = LogisticRegression( class_weight = 'balanced')
gs = GridSearchCV(
    estimator=lr,  
    param_grid=params,  
    cv=kf,  
    error_score=1,  
    scoring='roc_auc',  
    n_jobs=n_jobs,
    verbose=2,
)
gs.fit( X= X_train_var, y= y_train_var)
best_score = gs.best_score_
best_lr_var = gs.best_estimator_
print(best_score)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:  1.7min
[Parallel(n_jobs=3)]: Done  75 out of  75 | elapsed:  3.3min finished


0.720332661255
Wall time: 3min 22s


In [37]:
y_pred_var = best_lr_var.predict(X_test_var)
score_valid = roc_auc_score(y_pred_var, y_test_var)
score_valid

0.5981979001485549

In [41]:
%%time
# train bagging at logreg
parameters = {'max_features': [0.5, 0.7, 0.9, 1.], 'max_samples': [0.5, 0.7, 0.9, 1.], "base_estimator__C": [0.05, 0.1, 1]}
skf = StratifiedKFold(shuffle=True, random_state=42)
lg = LogisticRegression(class_weight= 'balanced',random_state=42)
bg = BaggingClassifier(lg, random_state=42, n_estimators=100)
r_grid_search = RandomizedSearchCV(bg, parameters, scoring ='roc_auc', n_iter=10, cv=skf, random_state=1, n_jobs = n_jobs)
r_grid_search = r_grid_search.fit(train_code, target_code)
print(r_grid_search.best_score_)

0.728769187951
Wall time: 1h 42min 36s


In [52]:
best_bglg = r_grid_search.best_estimator_

In [43]:
y_bag_log_pred =  best_bglg.predict(test_code)
write_to_submission_file(y_bag_log_pred.astype(int), test_code.index, 'log_bag_submission.csv' )

In [50]:
%%time
parameters = {'max_features': ['log2',20, 30, 40],
              'min_samples_leaf': [ 3, 5], 
              'max_depth': [20, 25, 30],
               'oob_score': [True]}
etc = ExtraTreesClassifier(n_estimators=300, random_state=42, n_jobs= n_jobs, bootstrap= True)
etcv = GridSearchCV(etc, parameters, n_jobs=-1, cv=skf, verbose=1, scoring='roc_auc')
etcv.fit(X_train_code, y_train_code)
print(etcv.best_estimator_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 12.7min
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed: 22.7min finished


ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
           max_depth=30, max_features=20, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=3, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=3,
           oob_score=True, random_state=42, verbose=0, warm_start=False)
Wall time: 23min 19s


In [49]:
etcv.best_score_

0.73331790819055587

In [65]:
# train tuned model for ensemble
lg = LogisticRegression(class_weight= 'balanced',random_state=42, C=0.1)
bglg = BaggingClassifier(lg, random_state=42, n_estimators=400, bootstrap=True, max_features=0.9,
         max_samples=0.9,  n_jobs=n_jobs)
etc = ExtraTreesClassifier(n_estimators=400, random_state=42, n_jobs= n_jobs, bootstrap= True,
                           class_weight='balanced',max_depth=30, max_features=20, 
                           min_samples_leaf=3, min_samples_split=2,
                           oob_score=True)

In [66]:
%%time
bglg.fit(X_train_code, y_train_code)
etc.fit(X_train_code, y_train_code)

Wall time: 19min 41s


In [81]:
y_bglg_pred = bglg.predict_proba(X_test_code)
y_etc_pred = etc.predict_proba(X_test_code)
y_pred = [(etc_p[1]*0.4 + bglg_p[1]*0.6) for etc_p, bglg_p in zip(y_etc_pred, y_bglg_pred)]
y_pred = [ round(x) for x in y_pred]
print(roc_auc_score(y_test_code, y_pred))

0.681123909378


In [86]:
print('Bagging LogReg: ',roc_auc_score(y_test_code, bglg.predict(X_test_code)))

Bagging LogReg:  0.677340788694


In [88]:
print('Extra DT: ',roc_auc_score(y_test_code, etc.predict(X_test_code)))

Extra DT:  0.653590380931


In [ ]:
bglg.fit(train_code, target_code)
etc.fit(train_code, target_code)
y_bglg_pred = bglg.predict_proba(test_code)
y_etc_pred = etc.predict_proba(test_code)
y_pred = [(etc_p[1]*0.4 + bglg_p[1]*0.6) for etc_p, bglg_p in zip(y_etc_pred, y_bglg_pred)]
y_pred = [ round(x) for x in y_pred]

In [62]:
write_to_submission_file(np.asarray(y_pred).astype(int), test_code.index, 'simple_ensemble_submission.csv' )
write_to_submission_file(np.asarray(etc.predict(test_code)).astype(int), test_code.index, 'etc_submission.csv' )
write_to_submission_file(np.asarray(bglg.predict(test_code)).astype(int), test_code.index, 'bglg_submission.csv' )